# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import csv

# set resources paths
url_school = "https://raw.githubusercontent.com/marazap90/Pandas_HW/master/Resources/schools_complete"
url_student = "https://raw.githubusercontent.com/marazap90/Pandas_HW/master/Resources/students_complete"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(url_school)
student_data = pd.read_csv(url_student)

In [2]:
# Combine the data into a single dataset
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# better formatting: 

# volumes:
def volume(number):
    return "{0:,d}".format(number)


#convert large values to currency for better visualization
def currency(number):
    return "${0:,d}".format(number)

# percentage
def percent(number):
    return "{:,.2%}".format(number)

In [4]:
# Calculate the total number of schools
print("Total number of schools:", len(df["school_name"].unique()))

# Calculate the total number of students
students = volume(len(df["student_name"].unique()))
print("Total number of students:", students)

# Calculate the total budget
usd_budget = currency(school_data["budget"].sum())
print("Total number of students:", usd_budget)

# Calculate the average math score 
math = percent(df["math_score"].mean()/100)
print("Average math score:", math)

# Calculate the average reading score
read = percent(df["reading_score"].mean()/100)
print("Average reading score:", read)

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall = percent( ( (df["math_score"].mean() + df["reading_score"].mean()) / 2) /100)
print("Overall average score:", overall)

# Calculate the percentage of students with a passing math score (70 or greater)
pass_math = percent(len(df[df['math_score'] >= 70]) / len(df))
print("Percentage of students with a passing math score:", pass_math)

# Calculate the percentage of students with a passing reading score (70 or greater)
pass_read = percent(len(df[df['reading_score'] >= 70]) / len(df))
print("Percentage of students with a passing reading score:", pass_read)

# Optional: give the displayed data cleaner formatting

Total number of schools: 15
Total number of students: 32,715
Total number of students: $24,649,428
Average math score: 78.99%
Average reading score: 81.88%
Overall average score: 80.43%
Percentage of students with a passing math score: 74.98%
Percentage of students with a passing reading score: 85.81%


In [5]:
# Create a dataframe to hold the above results
district_dict = {
    "Total number of schools" : len(df["school_name"].unique()),
    "Total number of students" : volume(len(df["student_name"].unique())),
    "Average math score" : percent(df["math_score"].mean()/100),
    "Average reading score" : percent(df["reading_score"].mean()/100),
    "Overall average score" : percent( ( (df["math_score"].mean() + df["reading_score"].mean()) / 2) /100),
    "Percentage of students with a passing math score" : percent(len(df[df['math_score'] >= 70]) / len(df)),
    "Percentage of students with a passing reading score:" : percent(len(df[df['reading_score'] >= 70]) / len(df)),   
}

In [6]:
summary_district = pd.DataFrame.from_dict(district_dict, orient="index").T
summary_district

,Total number of schools,Total number of students,Average math score,Average reading score,Overall average score,Percentage of students with a passing math score,Percentage of students with a passing reading score:
0,15,"32,715",78.99%,81.88%,80.43%,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [7]:
# get school type from school df only
s_type = school_data.set_index(["school_name"])["type"]

In [8]:
# Get totals

stu_per_school = df["school_name"].value_counts()

# create a group by df on school name 
groupby_df = df.groupby("school_name")
s_budgets = groupby_df["budget"].mean()
budget_p_stu = s_budgets / stu_per_school


# calculate averages per school
s_avg_math = round(groupby_df["math_score"].mean(),2)
s_avg_reading = round(groupby_df["reading_score"].mean(),2)
s_avg_overall = round((s_avg_math + s_avg_reading) / 2,2)


# % students who passed in math by school
math_pass = df[(df["math_score"]>69)]
s_math_pass = math_pass.groupby(["school_name"]).count()["student_name"]
s_math_pass_rate = (s_math_pass / stu_per_school)

# % students who passed in reading by school
reading_pass = df[(df["reading_score"]>69)]
s_reading_pass = reading_pass.groupby(["school_name"]).count()["student_name"]
s_reading_pass_rate = s_reading_pass / stu_per_school

s_overall_pass_rate = round((s_math_pass_rate + s_reading_pass_rate) / 2,2)

In [9]:
# Dictionary for schoool summary  
summary_school_dict = {
    "School Type":s_type,
    "Number of Students":stu_per_school,
    "Budget per School":s_budgets,
    "Budget per Student":budget_p_stu,
    "Average Math Score":s_avg_math,
    "Average Reading Score":s_avg_reading,
    "Overall Score":s_avg_overall,
    "Math Passing Rate":s_math_pass_rate,
    "Reading Passing Rate":s_reading_pass_rate,
    "Overall Passing Rate":s_overall_pass_rate
}

# save a raw/unformatted version for later use 
raw_summary_school_df = pd.DataFrame.from_dict(summary_school_dict,orient="columns")

In [10]:
summary_school_df = pd.DataFrame.from_dict(summary_school_dict,orient="columns")
summary_school_df["Budget per School"] = summary_school_df["Budget per School"].map("${:,.0f}".format)
summary_school_df["Budget per Student"] = summary_school_df["Budget per Student"].map("${:,.0f}".format)
summary_school_df["Math Passing Rate"] = (summary_school_df["Math Passing Rate"]*100).map("{:.0f}%".format)
summary_school_df["Reading Passing Rate"] = (summary_school_df["Reading Passing Rate"]*100).map("{:.0f}%".format)
summary_school_df["Overall Passing Rate"] = (summary_school_df["Overall Passing Rate"]*100).map("{:.0f}%".format)

summary_school_df

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,Overall Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,79.04,67%,82%,74%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,83.52,94%,97%,96%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,78.94,66%,81%,73%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,78.92,68%,79%,74%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,83.58,93%,97%,95%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,79.11,67%,81%,74%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,83.80,93%,96%,94%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,78.90,66%,81%,74%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,79.02,66%,81%,74%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,83.94,95%,96%,95%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [11]:
top_schools = summary_school_df.sort_values(by="Overall Passing Rate",ascending=False)
top_schools.head(5)

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,Overall Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,83.52,94%,97%,96%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,83.58,93%,97%,95%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,83.94,95%,96%,95%
Shelton High School,Charter,1761,"$1,056,600",$600,83.36,83.73,83.54,94%,96%,95%
Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,83.64,93%,97%,95%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [12]:
worst_schools = summary_school_df.sort_values(by="Overall Passing Rate",ascending=True)
worst_schools.head(5)

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,Overall Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,78.94,66%,81%,73%
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,78.79,66%,80%,73%
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,79.04,67%,82%,74%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,78.92,68%,79%,74%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,79.11,67%,81%,74%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [13]:
# Get average math score per grade using student_data df
math_9 = student_data.loc[student_data['grade'] == '9th'].groupby("school_name")["math_score"].mean()
math_10 = student_data.loc[student_data['grade'] == '10th'].groupby("school_name")["math_score"].mean()
math_11 = student_data.loc[student_data['grade'] == '11th'].groupby("school_name")["math_score"].mean()
math_12 = student_data.loc[student_data['grade'] == '12th'].groupby("school_name")["math_score"].mean()

# dict to store math averages per grade
math_grades_dict = {
    "9th":math_9,
    "10th":math_10,
    "11th":math_11,
    "12th":math_12   
}

# create df from dict above, format numbers to one decimal
math_per_grade_df = pd.DataFrame.from_dict(math_grades_dict)
math_per_grade_df["9th"] = math_per_grade_df["9th"].map("{:,.1f}".format)
math_per_grade_df["10th"] = math_per_grade_df["10th"].map("{:,.1f}".format)
math_per_grade_df["11th"] = math_per_grade_df["11th"].map("{:,.1f}".format)
math_per_grade_df["12th"] = math_per_grade_df["12th"].map("{:,.1f}".format)
math_per_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [14]:
# Get average reading score per grade using student_data df
reading_9 = student_data.loc[student_data['grade'] == '9th'].groupby("school_name")["reading_score"].mean()
reading_10 = student_data.loc[student_data['grade'] == '10th'].groupby("school_name")["reading_score"].mean()
reading_11 = student_data.loc[student_data['grade'] == '11th'].groupby("school_name")["reading_score"].mean()
reading_12 = student_data.loc[student_data['grade'] == '12th'].groupby("school_name")["reading_score"].mean()

# dict to store reading averages per grade
reading_grades_dict = {
    "9th":reading_9,
    "10th":reading_10,
    "11th":reading_11,
    "12th":reading_12   
}

# create df from dict above, format numbers to one decimal
reading_per_grade_df = pd.DataFrame.from_dict(reading_grades_dict)
reading_per_grade_df["9th"] = reading_per_grade_df["9th"].map("{:,.1f}".format)
reading_per_grade_df["10th"] = reading_per_grade_df["10th"].map("{:,.1f}".format)
reading_per_grade_df["11th"] = reading_per_grade_df["11th"].map("{:,.1f}".format)
reading_per_grade_df["12th"] = reading_per_grade_df["12th"].map("{:,.1f}".format)
reading_per_grade_df 

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [15]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
budget_bins = ["<$585", "$585-615", "$615-645", "$645-675"]

In [16]:
# create df for data from "budget per student" using bins
score_by_budget = raw_summary_school_df[[
    "Average Math Score", 
    "Average Reading Score", 
    "Math Passing Rate", 
    "Reading Passing Rate", 
    "Overall Passing Rate"
]].groupby(pd.cut(raw_summary_school_df["Budget per Student"], bins=spending_bins, labels=budget_bins)).mean()

score_by_budget["Average Math Score"] = score_by_budget["Average Math Score"].map("{:,.2f}".format)
score_by_budget["Average Reading Score"] = score_by_budget["Average Reading Score"].map("{:,.2f}".format)
score_by_budget["Math Passing Rate"] = score_by_budget["Math Passing Rate"].map("{:,.2f}".format)
score_by_budget["Reading Passing Rate"] = score_by_budget["Reading Passing Rate"].map("{:,.2f}".format)
score_by_budget["Overall Passing Rate"] = score_by_budget["Overall Passing Rate"].map("{:,.2f}".format)
score_by_budget

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
Budget per Student,,,,,
<$585,83.45,83.94,0.93,0.97,0.95
$585-615,83.60,83.89,0.94,0.96,0.95
$615-645,79.08,81.89,0.76,0.86,0.81
$645-675,77.00,81.03,0.66,0.81,0.74


## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
school_bins = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [18]:
# create df for data from "Number of Students" using bins
score_by_size = raw_summary_school_df[[
    "Average Math Score", 
    "Average Reading Score", 
    "Math Passing Rate", 
    "Reading Passing Rate", 
    "Overall Passing Rate"
]].groupby(pd.cut(raw_summary_school_df["Number of Students"], bins=size_bins, labels=school_bins)).mean()

score_by_size["Average Math Score"] = score_by_size["Average Math Score"].map("{:,.2f}".format)
score_by_size["Average Reading Score"] = score_by_size["Average Reading Score"].map("{:,.2f}".format)
score_by_size["Math Passing Rate"] = score_by_size["Math Passing Rate"].map("{:,.2f}".format)
score_by_size["Reading Passing Rate"] = score_by_size["Reading Passing Rate"].map("{:,.2f}".format)
score_by_size["Overall Passing Rate"] = score_by_size["Overall Passing Rate"].map("{:,.2f}".format)
score_by_size

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
Number of Students,,,,,
Small (<1000),83.82,83.93,0.94,0.96,0.94
Medium (1000-2000),83.37,83.87,0.94,0.97,0.95
Large (2000-5000),77.74,81.34,0.70,0.83,0.76


## Scores by School Type

* Perform the same operations as above, based on school type.

In [19]:
# create bins
type_bins = [0,1,2]

# name bins
type_bin_names = ["Charter","District"]

In [20]:
# create df for data from School Type" using bins
# replace Charter and District with 1 and 2 to do bins
score_by_type = raw_summary_school_df
score_by_type["School Type"] = score_by_type["School Type"].replace({"Charter": 1, "District":2})

score_by_type = raw_summary_school_df[[
    "Average Math Score", 
    "Average Reading Score", 
    "Math Passing Rate", 
    "Reading Passing Rate", 
    "Overall Passing Rate"
]].groupby(pd.cut(raw_summary_school_df["School Type"], bins=type_bins, labels=type_bin_names)).mean()

score_by_type["Average Math Score"] = score_by_type["Average Math Score"].map("{:,.2f}".format)
score_by_type["Average Reading Score"] = score_by_type["Average Reading Score"].map("{:,.2f}".format)
score_by_type["Math Passing Rate"] = score_by_type["Math Passing Rate"].map("{:,.2f}".format)
score_by_type["Reading Passing Rate"] = score_by_type["Reading Passing Rate"].map("{:,.2f}".format)
score_by_type["Overall Passing Rate"] = score_by_type["Overall Passing Rate"].map("{:,.2f}".format)
score_by_type

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,0.94,0.97,0.95
District,76.96,80.97,0.67,0.81,0.74
